Testing FNO

In [1]:
%load_ext autoreload
%autoreload 2

# Define autroreload so that it doesn't cause pain in the ass when we change the functions and run this notebook

In [2]:
import sys
from pathlib import Path
project_root = Path.cwd().resolve().parents[2]
github_root = Path.cwd().resolve().parents[1]
sys.path.append(str(project_root))
sys.path.append(str(github_root))

print(project_root)
print(github_root)

C:\SenkDosya\Projects\AeroML
C:\SenkDosya\Projects\AeroML\FlowField


In [3]:
from FlowField.scripts.auxiliary import avoid_MKL_bug


cfg_fno = {
    'model_type': 'FNO_nD',
    'cfg_model': {
        'cfg_p': {
       'in_channels': 1,
        'out_channels': 64,
        'hidden_channels': [16,8],
        'dropout': 0.05,
        'act_fn': 'gelu' 
        },
        'cfg_q': {
            'in_channels': 64,
            'out_channels': 1,
            'hidden_channels': [8,16],
            'dropout': 0.05,
            'act_fn': 'gelu'
        },
        'fb_hidden_channels': [32,32],
        'fb_modes': [[25,25],[25,25],[25,25]],
        'fb_kernel': [1,1,1],
        'fb_factorization': None
    }
}

avoid_MKL_bug(cfg_fno['model_type'])

In [4]:
from FlowField.scripts.loaders import load_model

fno = load_model(cfg_fno)

In [5]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

fno.to(device)


cuda


FNO_nD(
  (P_Net): ChannelMLP(
    (act_fn): GELU(approximate='none')
    (dropout): ModuleList(
      (0-1): 2 x Dropout(p=0.05, inplace=False)
    )
    (conv1d_layers): ModuleList(
      (0): Conv1d(1, 16, kernel_size=(1,), stride=(1,))
      (1): Conv1d(16, 8, kernel_size=(1,), stride=(1,))
      (2): Conv1d(8, 64, kernel_size=(1,), stride=(1,))
    )
  )
  (Q_Net): ChannelMLP(
    (act_fn): GELU(approximate='none')
    (dropout): ModuleList(
      (0-1): 2 x Dropout(p=0.05, inplace=False)
    )
    (conv1d_layers): ModuleList(
      (0): Conv1d(64, 8, kernel_size=(1,), stride=(1,))
      (1): Conv1d(8, 16, kernel_size=(1,), stride=(1,))
      (2): Conv1d(16, 1, kernel_size=(1,), stride=(1,))
    )
  )
  (FourierBlock): Sequential(
    (0): FourierBlockND(
      (SpectralConv): SpectralConvolutionND(
        (einsum): compl_mulnd()
      )
      (LocalLinear): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
      (norm1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, 

In [6]:
seq_len = 225
batch = 10
ch = 1

x = torch.rand(batch, ch, seq_len, seq_len, dtype=torch.float32, device=device)

In [7]:
pred= fno(x)